In [15]:
import os
import tqdm
import h5py
import xarray as xr
from dask.distributed import Client
from DASStore.modules import convert

In [16]:
import numpy as np
import pandas as pd
import datetime
from scipy.signal import detrend
from numpy.fft import fftshift, fft2, fftfreq
from datetime import datetime as DT
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [17]:
def get_sampling_rate(file_base_path, 
                      chan_min=0,
                      chan_max=-1, 
                      verbose=False,
                      pad=False):
#     data = np.array([])
#     time = np.array([])
    collecting_samplerate = []

        
    dt = datetime.timedelta(minutes=1) # Assume one minute file duration
    file_list = os.listdir(file_base_path)
    
    for i in range(len(file_list)):
        if file_list[i].endswith(".h5"):
            this_file = h5_files_path + file_list[i]
        
            try:
                f = h5py.File(this_file,'r')
                attrs = dict(f['Acquisition'].attrs)
                collecting_samplerate.append(attrs['PulseRate'])

            except Exception as e: 
                print('File problem with: %s'%this_file)
                print(e)
    
    return collecting_samplerate

In [18]:
h5_files_path = '/Volumes/Elements/'
# h5_files_path = '/Users/stlp/Desktop/samplefiles/'

In [20]:
# os.listdir(h5_files_path)

# Creating the pandas dataframe

Create a csv file
* of all file names
* dimensions in time and distance
* sampling rate


In [21]:
File_Name = []
Time_dimension = []
Dist_dimension = []

file_list = os.listdir(h5_files_path)

In [22]:
for i in range(len(file_list)):
    if file_list[i].endswith(".h5"):
        File_Name.append(file_list[i])
        file = h5py.File(h5_files_path+file_list[i], 'r')
        Time_dimension.append(file['Acquisition']['Raw[0]']['RawData'].shape[0])
        Dist_dimension.append(file['Acquisition']['Raw[0]']['RawData'].shape[1])

In [23]:
Sampling_Rate = get_sampling_rate(file_base_path = h5_files_path)

In [24]:
data = {'File_Name': File_Name,
        'Time_dimension': Time_dimension,
        'Dist_dimension': Dist_dimension,
        'Sampling_Rate': Sampling_Rate}

In [25]:
DAS_file_details = pd.DataFrame(data)

In [26]:
DAS_file_details.head()

,File_Name,Time_dimension,Dist_dimension,Sampling_Rate
0,whidbey_2022-10-19_05-52-00_UTC_000469.h5,30000,1721,500.0
1,whidbey_2022-10-26_19-28-00_UTC_011360.h5,120000,1721,2000.0
2,whidbey_2022-10-22_09-25-00_UTC_004998.h5,120000,1721,2000.0
3,whidbey_2022-10-29_21-22-00_UTC_015793.h5,6000,1721,100.0
4,whidbey_2022-10-29_11-49-00_UTC_015220.h5,6000,1721,100.0


In [27]:
DAS_file_details.shape

(15830, 4)

In [28]:
DAS_file_details.to_csv('DAS_file_details.csv', index = False)